In [1]:
import yfinance as yf
from datetime import datetime as dt 

import pandas as pd
import numpy as np

from scipy.stats import norm

In [55]:
chosen_date = '2025-07-18'
ticker = '^SPX'
spx = yf.Ticker(ticker)

expirations = spx.options
expirations = [expiry for expiry in expirations if \
               (dt.strptime(expiry, '%Y-%m-%d') - dt.strptime(chosen_date, '%Y-%m-%d')).days >= 40
               and (dt.strptime(expiry, '%Y-%m-%d') - dt.strptime(chosen_date, '%Y-%m-%d')).days <= 270]

In [56]:
market_price = {}

for expiry in expirations:
    opt_chain = spx.option_chain(expiry)
    calls = opt_chain.calls[["strike", "lastPrice"]]
    
    market_price[expiry] = {}
    market_price[expiry]['Strike'] = calls['strike']
    market_price[expiry]['Price'] = calls['lastPrice']

all_strikes = [v['Strike'] for i, v in market_price.items()]
common_strikes = set.intersection(*map(set, all_strikes))
common_strikes = sorted(common_strikes)

In [104]:
# Filter and organize the data.
prices = []
maturities = []

for date, v in market_price.items():
    maturities.append((dt.strptime(date, '%Y-%m-%d') - dt.today()).days / 365)
    price = [v['Price'][i] for i, x in enumerate(v['Strike']) if x in common_strikes]
    prices.append(price)

In [105]:
for date, v in market_price.items():
    maturities.append((dt.strptime(date, '%Y-%m-%d') - dt.today()).days / 365.25)
    price = [v['Price'][i] for i, x in enumerate(v['Strike']) if x in common_strikes]
    prices.append(price)
 
 # Convert in DataFrame
price_arr = np.array(prices, dtype=object)
np.shape(price_arr)

(32, 22)

In [106]:
# maturities = ... # ton array/list trié
maturities_sorted = np.array(sorted(maturities))
n_desired = 20  # combien tu veux en sélectionner

indices = np.linspace(0, len(maturities_sorted)-1, n_desired, dtype=int)
selected_maturities = maturities_sorted[indices]

common_strikes = np.array(common_strikes)

In [107]:
priceSurface = pd.DataFrame(price_arr, index=maturities, columns=common_strikes)
priceSurface = priceSurface.iloc[priceSurface.index.isin(selected_maturities), (priceSurface.columns < 7000)]
priceSurface

,5000.0,6000.0,6150.0,6175.0,6200.0,6225.0,6250.0,6275.0,6300.0,6325.0,6350.0,6400.0,6450.0,6500.0,6550.0,6650.0,6700.0,6750.0,6800.0,6900.0
0.109589,1307.31,370.98,247.82,212.64,196.59,177.37,161.01,144.03,129.73,113.53,99.68,73.72,54.14,38.06,25.65,10.71,6.7,4.2,2.62,1.35
0.128767,1299.41,356.96,260.0,219.59,209.29,165.39,179.97,134.43,148.1,123.92,110.2,85.0,61.7,44.89,32.0,14.21,9.25,5.95,4.25,1.74
0.243836,1350.81,435.05,328.26,286.24,280.98,239.74,245.94,231.88,214.65,192.48,181.15,151.78,124.14,101.2,81.7,50.5,38.39,29.0,24.1,11.93
0.282192,1033.5,459.03,352.72,325.06,313.05,286.25,268.3,252.27,234.9,212.66,203.37,161.7,142.97,121.16,94.01,60.2,51.29,40.49,30.96,18.11
0.339726,1380.47,488.75,371.0,352.82,331.23,297.05,297.8,281.8,266.3,247.17,232.67,201.1,169.31,146.89,127.85,86.25,68.3,56.7,45.6,28.63
0.416438,1399.66,522.03,413.9,387.3,367.48,331.53,335.57,314.5,300.82,283.9,267.2,237.26,208.59,181.0,156.1,113.95,95.91,80.32,66.73,44.65
0.449315,1407.96,535.8,406.09,389.93,383.9,333.22,349.29,320.25,314.04,297.28,279.1,251.82,219.21,192.63,176.57,124.82,110.86,89.04,75.02,51.77
0.493151,1399.45,559.31,441.9,434.92,407.2,401.6,369.1,358.7,338.27,320.2,303.7,272.0,242.0,214.0,194.5,140.55,122.08,104.15,88.82,62.5
0.695890,1351.43,644.39,530.22,484.2,488.32,460.02,434.34,433.44,416.45,406.92,393.59,358.26,330.51,301.92,262.26,220.18,191.18,168.66,149.69,115.27
0.742466,1506.92,636.2,548.87,529.75,488.88,485.44,464.72,464.92,425.25,414.14,404.9,349.6,340.66,295.62,287.45,238.4,206.71,188.41,165.62,129.17


In [110]:
data_price = priceSurface.melt(ignore_index=False).reset_index()
data_price.columns = ["Maturity", "Strike", "Price"]

In [6]:
def bs_call_price(S, K, T, r, sigma):
    if T == 0 or sigma == 0:
        return max(S - K, 0)
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

def vega(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def implied_vol_call(price, S, K, T, r, tol=1e-8, max_iter=100):
    sigma = 0.2
    for i in range(max_iter):
        price_est = bs_call_price(S, K, T, r, sigma)
        vega_est = vega(S, K, T, r, sigma)
        if vega_est < 1e-8:  
            break
        incr = (price_est - price) / vega_est
        sigma = sigma - incr
        if abs(incr) < tol:
            return sigma
    return np.nan  


In [10]:
IV_list = []
for idx, row in data_price.iterrows():
    S = 6296.79
    K = row['Strike']
    T = row['Maturity']
    r = 0.0408
    price = row['Price']
    try:
        iv = implied_vol_call(price, S, K, T, r)
    except Exception as e:
        iv = np.nan
    IV_list.append(iv)

data_price['IV'] = IV_list

C:\Users\baris\AppData\Local\Temp\ipykernel_36600\1803841054.py:4: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
C:\Users\baris\AppData\Local\Temp\ipykernel_36600\1803841054.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))


In [35]:
data_price.to_csv(r'C:\Users\baris\Desktop\Master Thesis\5. Python\Data\Option_Data_SPX_18072025.csv', index=False)